# MNIST 手寫數字辨識 (MNIST_CNN_TF)

2017/07/16  
Jay Hsu

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
- 好用的markdown語法
[markdown](https://www.zybuluo.com/codeep/note/163962#1如何输入一个方程式序列)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas
- import lzma error: 請用python 3


## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. 開始Deep Learning - CNN](#開始Deep Learning)  
-  [3. 補充資料](#補充資料)
-  [4. Homework](#Homework)

<a id='import'></a>
## Import Package & Functions

In [6]:
import os
import sys
import numpy as np
from PIL import Image
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tensorflow.python.client import device_lib
import pandas as pd

In [7]:
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))

In [8]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [9]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data
#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [10]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')
    
print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])

list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [11]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [12]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='開始Deep Learning'></a>
## 2. 開始Deep Learning - CNN
- CNN Motivation: 用來做影像辨識  
<br>  
- 影像辨識的3個重要的性質(Property) 
  - Property 1: 重要的Pattern通常出現在圖片的一小部分, 而不是整張圖片
  - Property 2: 同樣的Pattern會出現在圖片的不同區域
  - Property 3: 對圖片做subsampling而不會影響圖片的辨識結果  
<br>  
- CNN的三個重要功能/操作 
  - Convolution 
    - Filter: 用來detect圖像上的小區域pattern, 當detect到pattern時output的值就會特別大, 一張圖片上可能有很多個區域都有pattern 
    - Stride: Filter一次在圖像上移動幾個pixel
    - padding: 為了維持Filter後的圖片大小, 可以在圖片周圍補0
    - CNN其實是Fully Connect的簡化版, 使用了較少的parameter
  - Max pooling
    - group matrix裡的element
    - 每一個group選一個最大的
    - 不一定要max, 也可以average
  - Flatten
    - 把image形狀拉成直的
    - 目的是為了做最後的分類  
<br>   
- Overview
  - matrix calculation process
  - parameter qty in each layer  
<br>  
- some tips: 
  - 靠近input的layer偵測比較單純的pattern, 所以channel設定較小
  - 靠近output的layer偵測比較抽象的pattern, 所以channel設定較大  
<br>
- 參考圖片：
![](img/cnn_1.gif)
![](img/cnn_2.png)

- Deep Learning 3 Steps 
    - Define Function Set
        -  [A. 定義參數](#定義參數) 
        -  [B. 設計一個Model從X預測Y](#設計一個) 
    - Evaluate goodness of functions
        -  [C. 選一個loss function,](#選一個loss) 
        -  [D. 選一個optimizer](#選一個o) 
    - Choose the best function
        -  [E. 開始執行訓練](#開始執行) 
        -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [49]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, 784], name=X'')
Y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [48]:
# 設定 weight 和 bais
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    W = tf.Variable(initial)
    variable_summaries(W)
    return W
def bias_variable(shape):
    initial = tf.constant(value=0.1, shape=shape)
    B = tf.Variable(initial)
    variable_summaries(B)
    return B
# 設定 cnn 的 layers
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME',use_cudnn_on_gpu=True)
    #stride:每次移動步伐, padding:在圖片的四周補0, SAME就是補到conv後要和原圖一樣大張
def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    #ksize:框框大小, stride:每次移動步伐

<a id='設計一個'></a>
###  B. 設計一個Model從X & X history預測Y  
Input Layer: X  
Hidden Layer 1: $H_1=pooling(f(W_1(X)+B_1)$  
Hidden Layer 2: $H_2=pooling(f_2(W_2H_1+B_2))$  
Output Layer : $Y=pooling(f_3(W_3H_2+B_3))$  


### L1. Input Layer 輸入層
- do nothing

### L2. Hidden Layer 隱藏層 x 3

**  Fisrt Convolution + Max Pooling layer**

In [15]:
with tf.name_scope('conv1'):
    ## variables
    W_conv1 = weight_variable([3,3,1,32]) # filter_height, filter_width, in_channels, out_channels
    b_conv1 = bias_variable([32])
    ## build the layer
    X_image = tf.reshape(X, [-1, 28, 28,1]) # batch, in_height, in_width, in_channels
    h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)


** Second Convolution + Max Pooling layer**

In [16]:
with tf.name_scope('conv2'):
    ## variables
    W_conv2 = weight_variable([3,3,32,64])
    b_conv2 = bias_variable([64])
    ## build the layer
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

** fully-connected layer **

In [17]:
with tf.name_scope('full'):
    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

### L3. Output Layer 輸出層

In [18]:
#with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout
with tf.name_scope('output'):
    W_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])
    _pred = tf.matmul(h_fc1_drop, W_fc2)+b_fc2
    pred = tf.nn.softmax(_pred, name="pred")

with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1), name="correction")
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name="accuracy")

<a id='選一個loss'></a>
###  C. 選一個loss function, 當作Machine learning的目標
- cross_entorpy $-log(\Pr(Y_{true}))$

In [19]:
loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=_pred))

<a id='選一個o'></a>
### D. 選一個optimizer, 根據Data和我們訂的目標找參數W, B

#### Adam Optimizer
- 上週我們介紹的Adaptive gradient descent (Adagrad), 目的是讓gradient隨著時間衰減(time decay)
- 後來有人改良了Adagrad的公式, 就變成了$\color{blue}{RMPprop}$, 一樣是讓gradient隨時間衰減, 但效能較好
- 然後又有人覺得optimizer不應該只和目前這個時間點的gradient有關, 也應該和前幾個時間點的gradient有關, 就像慣性的概念一樣, 所以就發明了$\color{blue}{Momentun}$這個概念
- 把Adagrad加上改成RMPProp再加上Momentun就是$\color{blue}{Adam}$
- 詳細的介紹請參閱ML Lecture 9的影片

In [20]:
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

<a id='開始執行'></a>
### E. 開始執行訓練(Training Data + Validataion Data)

In [21]:
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
with tf.name_scope('performance'):  
    loss_scar = tf.summary.scalar('loss', loss)
    # Create a summary to monitor accuracy tensor
    acc_scar = tf.summary.scalar('accuracy', accuracy)
    # Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [22]:
sess = tf.InteractiveSession()
sess.run(init)

if not os.path.exists('tb-log_6'):
    os.mkdir('tb-log_6')
summary_writer_train = tf.summary.FileWriter('tb-log1/train',graph=tf.get_default_graph())
summary_writer_validation = tf.summary.FileWriter('tb-log1/validation',graph=tf.get_default_graph())

In [ ]:
#%%timeit -r 1 -n 1
epoch = 15
batch_size = 128
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        _, loss_s, acc_s, summary= sess.run([optimizer, loss_scar, acc_scar, merged_summary_op], feed_dict={X: batch_x, Y_:batch_y, keep_prob:0.5})
        summary_writer_train.add_summary(summary, ep * total_batch + i)
        if i%100 ==0:
            loss_v, acc_v, loss_s, acc_s, summary= sess.run([loss, accuracy, loss_scar, acc_scar, merged_summary_op], feed_dict={X: validation_X , Y_: validation_Y, keep_prob:1.0 })
            #acc = accuracy.eval({X: validation_X , Y_: validation_Y, keep_prob:1.0})
            #los=loss.eval({X: validation_X , Y_: validation_Y, keep_prob:1.0})
            summary_writer_validation.add_summary(summary, ep * total_batch + i)
            updateProgress('epoch:{x0}, batch:{x4} loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc_v,x3=loss_v,x4=i))
    print()


<a id='算一下正'></a>
### F. 算一下正確率Accuracy (用Testing Data)
- unseen data
- generalize from the training set to the test set
- 泛化

In [25]:
# predict all X and get the accuracy
_ = accuracy.eval({X: test_X , Y_: test_Y, keep_prob:1.0})
print('Accuracy:',_)

Accuracy: 0.1286


** Prediction **

In [26]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = pred.eval(feed_dict={X: train_X[21:30], keep_prob:1.0})
pred_tmp_df = pd.DataFrame(pred_tmp)
pred_tmp_df = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',pred_tmp_df.values)
showX(train_X[21:30])
print('answer: ',train_y[21:30])

print('第21~30筆資料的比對結果:',pred_tmp_df.values)

_ = correct_pred.eval({X: train_X[21:30] , Y_: train_Y[21:30], keep_prob:1.0})
print('第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [7 0 0 0 7 7 0 7 0]


answer:  [0 9 1 1 2 4 3 2 7]
第21~30筆資料的比對結果: [7 0 0 0 7 7 0 7 0]
第21~30筆資料的比對結果:        0
0  False
1  False
2  False
3  False
4  False
5  False
6  False
7  False
8  False


** 抓出所有預測錯的 **

In [ ]:
#sess.close()

<a id='補充資料'></a>
## 3. 補充資料

#### 參考一下Alpha Go的Network Structure
1. ** 應用前必須先思考, 同樣的Network架構是否適合用在你的Task上? Alpha GO沒有用Max Pooling就是一個最好的例子**  
2. ** Alpha Go的input layer將一張圖擴充成19*19*48的維度, 48表示每一個點都有48種feature(ex: 有沒有被黑子圍住, 有沒有被白子圍住, ....), 這就需要專業的Domain Knowledge來幫忙定義 **

![](img/alphago.png)

### 每一層的Filter到底在看些什麼？(or 什麼樣的圖片才能讓那一層的Filter output最大?)
https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html

<a id='Homework'></a>
## 4. Homework 
- 玩玩看CIFAR 10 Data set
- 李宏毅教授上課影片
- must  
    - [ML Lecture 11: Why Deep](https://www.youtube.com/watch?v=XsC9byQkUH8) : deep的好處, 有效率的使用data, required less parameter, 破除一些deep learning的迷思
- optional 
    - [ML Lecture 4: Classification](https://www.youtube.com/watch?v=fZAZUYEeIMg) : generative model 
    - [ML Lecture 5: Logistic regression](https://www.youtube.com/watch?v=hSXFuypLukA) : discriminative model
    - [ML Lecture 9: Tips for Deep Learning](https://www.youtube.com/watch?v=xki61j7z-30) : Adam的介紹
    - [ML Lecture 10: Convolution neural network](https://www.youtube.com/watch?v=FrKWiRv254g) 

***

In [45]:
save_model=True
restore_model = False

#### Save Model 

In [46]:
if save_model:
    saver = tf.train.Saver()
    save_path = saver.save(sess, "/model/cnn_model.ckpt")
    print('save cnn model done',save_path)

ValueError: At least two variables have the same name: conv2/W

** Restore Model **

In [44]:
if restore_model:
    sess=tf.Session()
    saver = tf.train.Saver()
    saver.restore(sess, "/model/cnn_model.ckpt")
    print('restore dnn model done')